<a href="https://colab.research.google.com/github/Hitika-Jain/LegalTalk/blob/main/notebooks/ocr_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!git clone https://github.com/Hitika-Jain/LegalTalk.git

%cd LegalTalk
#

Cloning into 'LegalTalk'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 23), reused 17 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 439.34 KiB | 8.97 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/LegalTalk/LegalTalk


In [2]:
pip install pandas

In [3]:
!pip install pymupdf nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 42.9 MB/s eta 0:00:00


In [8]:
!ls -R .

.:
data  graph-based-model.ipynb  requirements.txt  src  Untitled21.ipynb

./data:
case.pdf  statutes.csv

./src:
app.py		   graph_mapper.py   __pycache__      statute_ensemble.py
bert_mapper.py	   __init__.py	     regex_mapper.py  summarizer.py
explainability.py  preprocessing.py  statute_eda.py   utils.py

./src/__pycache__:
__init__.cpython-312.pyc  preprocessing.cpython-312.pyc


In [9]:
import sys
sys.path.append('/content/LegalTalk')

# verify
sys.path[-1]

'/content/LegalTalk'

In [5]:
import nltk
nltk.download('punkt_tab')

from src.preprocessing import parse_pdf_to_sentences

records = parse_pdf_to_sentences(
    "/content/drive/MyDrive/legal_dataset/Purushottam_Dashrath_Borate_vs_State_Of_Maharashtra_on_8_May_2015.PDF",
    out_path="/content/casetext.jsonl"
)

len(records), records[:3]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[Saved] 300 lines → /content/casetext.jsonl


(300,
 [{'page': 0,
   'line_id': 0,
   'start_char': 0,
   'end_char': 79,
   'text': 'Purushottam Dashrath Borate vs State Of Maharashtra on 8\nMay, 2015\nAuthor: H.L.'},
  {'page': 0,
   'line_id': 1,
   'start_char': 80,
   'end_char': 97,
   'text': 'Dattu\nBench: H.L.'},
  {'page': 0,
   'line_id': 2,
   'start_char': 98,
   'end_char': 458,
   'text': 'Dattu\n                                                             1\n                                                                              REPORTABLE\n                                         IN THE SUPREME COURT OF INDIA\n                                        CRIMINAL APPELLATE JURISDICTION\n                                        CRIMINAL APPEAL No.'}])

In [23]:
from src import regex_mapper

hits = regex_mapper.map_jsonl_file_to_hits(
    jsonl_path="/content/casetext.jsonl",
    out_csv="/content/regex_hits3.csv",
    include_legal_terms=True
)

len(hits)

Wrote 36 hits → /content/regex_hits3.csv


36

statute standarisation done in utils

checking the produced csv is correct or not

In [14]:
import pandas as pd, re
from collections import Counter

# load CSV you produced (update path if needed)
df = pd.read_csv("/content/regex_hits.csv", dtype=str).fillna('')

# A) Unique normalized statute tokens & counts
# Corrected column name from 'statute_normalized' to 'statute_raw'
counts = Counter(df['statute_raw'].tolist())
print("Unique statute tokens and counts:")
for k,v in counts.most_common():
    print(k, v)

# B) quick canonicalizer (convert 120-B -> IPC_120B, 376(2)(g) -> IPC_376_2_G)
def normalize_to_ipc(raw):
    if not raw: return None
    s = raw.upper().strip()
    s = re.sub(r'^(SECTION|SEC|S)\.?\s*', '', s)
    s = re.sub(r'[^0-9A-Z\(\)]', '', s)           # keep digits, letters, parentheses
    # handle (2)(g) -> _2_G
    s = re.sub(r'\)\(', '_', s)                   # join )(
    s = re.sub(r'[\(\)]', '_', s)                 # parens to underscores
    s = re.sub(r'_+', '_', s).strip('_')
    s = s.replace('-', '')                        # 120-B -> 120B
    return "IPC_" + s

# Map and show sample
# Corrected column name from 'statute_normalized' to 'statute_raw'
df['canonical_id'] = df['statute_raw'].apply(normalize_to_ipc)
print("\nSample mappings:")
# Corrected column name from 'statute_normalized' to 'statute_raw'
print(df[['statute_raw','match_text','canonical_id']].drop_duplicates().head(30))

Unique statute tokens and counts:
120-B 16
302 9
364 2
1860 1
313 1
376(2)(g) 1
404 1
366 1
43 1
376 1
380 1
1A 1

Sample mappings:
   statute_raw                              match_text canonical_id
0          302                            Sections 302      IPC_302
1        120-B                           Section 120-B     IPC_120B
2         1860                 Indian Penal Code, 1860     IPC_1860
3        120-B  Section 120-B of the Indian Penal Code     IPC_120B
4          302                             Section 302      IPC_302
5          302                  Section 302 of the IPC      IPC_302
6          364                            Sections 364      IPC_364
8        120-B                Section 120-B of the IPC     IPC_120B
12         313                             Section 313      IPC_313
16   376(2)(g)                       Section 376(2)(g)  IPC_376_2_G
18         364                             Section 364      IPC_364
20         404                             Section 4

weak label generation

In [24]:
import pandas as pd
from collections import defaultdict
from pathlib import Path

HITS_CSV = "/content/regex_hits3.csv"   # update to your CSV
OUT_WEAK = "/content/weak_labels.csv"

df = pd.read_csv(HITS_CSV, dtype=str).fillna('')
# build sentence key (page__line_id) to group
df['sent_key'] = df['page'].astype(str) + "__" + df['line_id'].astype(str)

grouped = defaultdict(set)
meta = {}
for _, row in df.iterrows():
    key = row['sent_key']
    cid = row.get('canonical_id')
    if pd.notna(cid) and cid and cid != 'None':
        grouped[key].add(cid)
    # keep a representative sentence text
    if key not in meta:
        meta[key] = {
            'page': row.get('page'),
            'line_id': row.get('line_id'),
            'sentence_text': row.get('sentence_text')
        }

rows = []
for k, ids in grouped.items():
    m = meta.get(k, {})
    rows.append({
        'sentence_id': k,
        'page': m.get('page',''),
        'line_id': m.get('line_id',''),
        'sentence_text': m.get('sentence_text',''),
        'canonical_statutes': ';'.join(sorted(ids))
    })

weak_df = pd.DataFrame(rows)
weak_df.to_csv(OUT_WEAK, index=False)
print("Wrote", OUT_WEAK, "rows:", len(weak_df))

# quick inspect
print(weak_df.head(10))
print("\nTop statutes (by weak-label frequency):")
stat_counts = {}
for s in weak_df['canonical_statutes']:
    for cid in s.split(';'):
        stat_counts[cid] = stat_counts.get(cid,0)+1
import itertools
for k,v in sorted(stat_counts.items(), key=lambda x:-x[1])[:30]:
    print(k, v)

Wrote /content/weak_labels.csv rows: 13
  sentence_id page line_id                                      sentence_text  \
0        0__8    0       8  By the impugned judgment and order,\nthe High ...   
1        2__5    2       5  Therefore, an offence under Section\n302 of th...   
2       2__31    2      31  On 05.03.2009, the\ncharges were framed under ...   
3       2__32    2      32  On 03.04.2010, the charge was altered and the ...   
4       2__33    2      33  Additionally, the charge under Section 120-B o...   
5       3__15    3      15  The Sessions Court has noticed that the eviden...   
6        4__5    4       5  After\naffording an opportunity of hearing to ...   
7        4__9    4       9  Aggrieved by the aforesaid judgment and order,...   
8       5__21    5      21  However, it would be apposite to notice the de...   
9        7__3    7       3  For instance, murder is of a person who had ac...   

                             canonical_statutes  
0                 

debugging where it can fail like line breaks and gaps

In [15]:
# Debugging regex mapper - run in Colab
import json, re, sys, os
from pathlib import Path

# Update these paths if different
JSONL = "/content/casetext.jsonl"   # path created by your parser
# Or try repo-local copy
if not Path(JSONL).exists():
    if Path("data/case_sentences.jsonl").exists():
        JSONL = "data/case_sentences.jsonl"
    elif Path("data/sample_sentences.jsonl").exists():
        JSONL = "data/sample_sentences.jsonl"
print("Using JSONL:", JSONL, "exists?", Path(JSONL).exists())

# 1) Print first N records from JSONL to confirm parser output
def show_first_records(jsonl_path, n=20):
    print(f"\n--- Showing up to {n} parsed records ---\n")
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= n:
                break
            try:
                rec = json.loads(line.strip())
            except Exception as e:
                print("JSON parse error on line", i, e)
                print(line)
                continue
            print(f"[{i}] page={rec.get('page')} line_id={rec.get('line_id')} chars=({rec.get('start_char')},{rec.get('end_char')})")
            txt = rec.get('text','').replace('\n',' ')
            # print a short preview
            print("   >", txt[:300])
            print()
    print("...done\n")

if Path(JSONL).exists():
    show_first_records(JSONL, n=30)
else:
    raise FileNotFoundError(f"JSONL file not found: {JSONL}")

# 2) Show the patterns we are using (from regex_mapper.build_statute_patterns)
from src import regex_mapper
patterns = regex_mapper.build_statute_patterns()
print("\n--- Compiled Patterns ---")
for name, pat in patterns:
    print(name, "->", pat.pattern)
print()

# 3) Run the matcher on each record and collect any matches
print("\n--- Running matcher over parsed sentences (first 200) ---\n")
matches_total = 0
no_match_but_keywords = []
with open(JSONL, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 200:
            break
        rec = json.loads(line)
        text = rec.get('text','') or ''
        found = regex_mapper.find_statutes_in_text(text, patterns=patterns)
        if found:
            matches_total += len(found)
            for m in found:
                print(f"[hit] line#{i} page={rec.get('page')} match={m['match_text']} -> norm={m['statute_norm']} pattern={m['pattern_name']}")
        else:
            # record sentences that contain likely keywords for further inspection
            low = text.lower()
            if 'section' in low or 'sec.' in low or 's.' in low or 'ipc' in low or 'article' in low or re.search(r'\b[0-9]{2,3}\b', low):
                no_match_but_keywords.append((i, text[:200]))
print("\nTotal matches found (first 200 lines):", matches_total)
print("Sentences with keywords but no match (sample up to 20):", len(no_match_but_keywords))
for idx, t in no_match_but_keywords[:20]:
    print(f" - line#{idx}: {t}")
print()

# 4) Fallback: run extremely broad patterns (debug only)
print("\n--- Fallback broad checks (debug) ---")
broad_patterns = {
    "section_simple": re.compile(r'\bsection\s+[:\s]*([0-9]{1,4}[A-Za-z\-]*)', re.IGNORECASE),
    "s_dot": re.compile(r'\bs\.\s*([0-9]{1,4}[A-Za-z\-]*)', re.IGNORECASE),
    "ipc_then_num": re.compile(r'\bipc[\s_:]*([0-9]{1,4}[A-Za-z\-]*)', re.IGNORECASE),
    "num_ipc_after": re.compile(r'\b([0-9]{1,4}[A-Za-z\-]*)\s*(?:,?\s*ipc)\b', re.IGNORECASE),
    "bare_num_letter": re.compile(r'\b([0-9]{2,3}[A-Za-z]?)\b')
}
broad_matches = 0
with open(JSONL, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 200:
            break
        rec = json.loads(line)
        text = rec.get('text','') or ''
        for pname, p in broad_patterns.items():
            m = p.search(text)
            if m:
                broad_matches += 1
                print(f"[broad:{pname}] line#{i} -> {m.group(0)} -> group1={m.group(1) if m.groups() else None}")
                break
print("Broad matches found (first 200):", broad_matches)

# 5) If 0 matches, produce a CSV report of sentences containing keywords for manual inspection
if matches_total == 0:
    import csv
    out_csv = "/content/regex_debug_keyword_lines.csv"
    with open(JSONL, 'r', encoding='utf-8') as fin, open(out_csv, 'w', newline='', encoding='utf-8') as fout:
        writer = csv.writer(fout)
        writer.writerow(["line_no", "page", "line_id", "text_snippet"])
        for i, line in enumerate(fin):
            rec = json.loads(line)
            text = rec.get('text','') or ''
            low = text.lower()
            if 'section' in low or 'sec.' in low or 's.' in low or 'ipc' in low or 'article' in low or re.search(r'\b[0-9]{2,3}\b', low):
                writer.writerow([i, rec.get('page'), rec.get('line_id'), text[:300].replace('\n',' ')])
    print(f"Wrote keyword lines report to {out_csv}. Please inspect this CSV to see why regex may have failed.")
else:
    print("Matches detected — no keyword report generated.")


Using JSONL: /content/casetext.jsonl exists? True

--- Showing up to 30 parsed records ---

[0] page=0 line_id=0 chars=(0,79)
   > Purushottam Dashrath Borate vs State Of Maharashtra on 8 May, 2015 Author: H.L.

[1] page=0 line_id=1 chars=(80,97)
   > Dattu Bench: H.L.

[2] page=0 line_id=2 chars=(98,458)
   > Dattu                                                              1                                                                               REPORTABLE                                          IN THE SUPREME COURT OF INDIA                                         CRIMINAL APPELLATE JURISDICTION

[3] page=0 line_id=3 chars=(459,531)
   > 1439 OF 2013                          PURUSHOTTAM DASHRATH BORATE & ANR.

[4] page=0 line_id=4 chars=(549,783)
   > …APPELLANT(                                                          VERSUS                          STATE OF MAHARASHTRA                                …RESPONDENT                                                  J U D G M E N 

diagnose if the script is running fine

In [16]:
# DIAGNOSTIC: test patterns on one sentence
import json
from pathlib import Path
from src import regex_mapper

JSONL = "/content/casetext.jsonl"
# pick a specific line that contained "Section 120-B" (from your report it was line#8 / line#70 / etc.)
target_indices = [8, 70, 99, 103]  # try several lines that the fallback reported
records = []
with open(JSONL, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i in target_indices:
            records.append((i, json.loads(line)))

patterns = regex_mapper.build_statute_patterns()
print("Testing compiled patterns:\n")
for idx, rec in records:
    text = rec.get('text','')
    print(f"--- line#{idx} preview: {text[:240]!r} ---")
    for name, pat in patterns:
        m = pat.search(text)
        print(f"{name:20s} -> match? {bool(m)}", f" | matched: {m.group(0)!r}" if m else "")
    print()


Testing compiled patterns:

--- line#8 preview: 'By the impugned judgment and order,\nthe High Court has confirmed the judgment of conviction and order of sentence passed by the Court\nof Sessions Judge, Pune in Sessions Case No.284 of 2008, dated 20.03.2012, whereby the learned\nSessions Ju' ---
section_keyword      -> match? True  | matched: 'Section 120-B'
ipc_prefix           -> match? False 
section_of_ipc       -> match? True  | matched: 'Section 120-B of the Indian Penal Code'
num_then_ipc         -> match? False 
article              -> match? False 
bare_number_with_letter -> match? False 

--- line#70 preview: 'On 03.04.2010, the charge was altered and the independent charge of conspiracy under\nSection 120-B of the IPC was added.' ---
section_keyword      -> match? True  | matched: 'Section 120-B'
ipc_prefix           -> match? False 
section_of_ipc       -> match? True  | matched: 'Section 120-B of the IPC'
num_then_ipc         -> match? False 
article              -> match

In [20]:
# === Debug + robust regex mapper update ===
import json, re, sys, os
from pathlib import Path

# 0) paths — adjust if needed
JSONL = "/content/casetext.jsonl"   # your parser output
OUT_CSV = "/content/regex_hits_debug.csv"

print("JSONL exists?", Path(JSONL).exists())
if not Path(JSONL).exists():
    raise FileNotFoundError(JSONL)

# 1) Inspect first record keys and a sample text (raw repr to reveal invisible chars)
with open(JSONL, 'r', encoding='utf-8') as f:
    first = json.loads(next(f))
print("\nFirst record keys:", list(first.keys()))
sample_text = first.get('text') or first.get('sentence') or first.get('line') or first.get('content')
print("\nSample text (repr):\n", repr(sample_text)[:500])
print("\nSample text (print):\n", sample_text[:500])

# Show character codes of first 120 chars to discover NBSP or zero-width
print("\nChar codes (first 120 chars):")
for i, ch in enumerate(sample_text[:120]):
    print(i, ch, hex(ord(ch)))

# 2) Load the regex code (we'll create a robust local copy for immediate use)
def build_patterns():
    flags = re.IGNORECASE | re.UNICODE
    gap = r'[\s\u00A0\-\:\.]*'
    patterns = [
        ("section_keyword", re.compile(r'\b(?:section|sec|s|secs|§)'+ gap + r'([0-9]{1,4}[A-Za-z\-\(\)\/]{0,10})\b', flags)),
        ("ipc_prefix", re.compile(r'\b(?:ipc|indian\s+penal\s+code)'+ gap + r'[^\w]{0,3}'+ gap + r'([0-9]{1,4}[A-Za-z\-\(\)\/]{0,10})\b', flags)),
        ("num_then_ipc", re.compile(r'\b([0-9]{1,4}[A-Za-z\-\(\)\/]{0,10})'+ gap + r'(?:,?\s*ipc|,?\s*indian\s+penal\s+code)\b', flags)),
        ("section_of_ipc", re.compile(r'\bsection'+ gap + r'([0-9]{1,4}[A-Za-z\-\(\)\/]{0,10})'+ gap + r'(?:of|under)'+ gap + r'(?:the\s*)?(?:ipc|indian\s+penal\s+code)\b', flags)),
        ("article", re.compile(r'\b(?:article|art)'+ gap + r'([0-9]{1,3}[A-Za-z\-\(\)\/]{0,6})\b', flags)),
        ("section_ipc_anywhere", re.compile(r'\bsection'+ gap + r'([0-9]{1,4}[A-Za-z\-\(\)\/]{0,10}).{0,20}?(?:ipc|indian\s+penal\s+code)\b', flags)),
        ("bare_number_with_letter", re.compile(r'(?<!\d)([0-9]{2,4}[A-Za-z])\b', flags)),
    ]
    return patterns

patterns = build_patterns()
print("\nLoaded patterns:", [p[0] for p in patterns])

# 3) Normalizer: replace NBSP with regular space, remove zero-width chars, collapse multiple spaces
_zero_width_chars = ['\u200b', '\u200c', '\u200d', '\ufeff']  # common zero width / BOM
def normalize_text_for_matching(s: str) -> str:
    if s is None:
        return ""
    s2 = s.replace('\u00A0', ' ')  # NBSP -> space
    for z in _zero_width_chars:
        s2 = s2.replace(z, '')
    # collapse weird multi-space runs
    s2 = re.sub(r'\s+', ' ', s2)
    return s2

# 4) Function to find matches on normalized text then map back to original indices
def find_matches_map_to_original(text: str, patterns=None):
    patterns = patterns or build_patterns()
    orig = text or ""
    norm = normalize_text_for_matching(orig)
    results = []
    # We'll keep track of a cursor in original text to find subsequent matches to avoid wrong repeats
    orig_cursor = 0
    for name, pat in patterns:
        for m in pat.finditer(norm):
            matched_norm = m.group(0)
            raw_group = m.group(1) if m.groups() else None
            # Try to find the matched_norm in the original text starting from orig_cursor
            # but matched_norm may have normalized spaces; try a few strategies:
            idx = orig.find(matched_norm, orig_cursor)
            if idx == -1:
                # strategy B: locate using raw_group if exists
                if raw_group:
                    # raw_group likely preserved (digits/letters). try to find raw_group in orig
                    idx = orig.find(raw_group, orig_cursor)
                    if idx != -1:
                        # compute match_end by adding len(raw_group) in orig
                        idx_end = idx + len(raw_group)
                        match_text_orig = orig[idx:idx_end]
                        statute_raw = raw_group
                    else:
                        # fallback: fuzzy by stripping spaces from both and search (last resort)
                        try:
                            mn_norm_nospace = re.sub(r'\s+', '', matched_norm)
                            orig_nospace = re.sub(r'\s+', '', orig)
                            pos_nospace = orig_nospace.find(mn_norm_nospace)
                            if pos_nospace != -1:
                                # estimate original index by mapping positions (approx)
                                # find approximate index by searching substring of length 10
                                glimpse = mn_norm_nospace[:10]
                                idx = orig.find(glimpse, orig_cursor)
                        except Exception:
                            idx = -1
                else:
                    idx = -1
            if idx == -1:
                # give up mapping to original for this match (skip)
                # but record debug info
                results.append({
                    'pattern': name,
                    'matched_normalized': matched_norm,
                    'raw_group': raw_group,
                    'mapped': False,
                    'orig_index': None,
                    'orig_match_text': None
                })
                continue
            # Success mapping to original
            # compute end based on matched_norm length vs original substring: try to match same length
            end = idx + len(matched_norm)
            # But matched_norm may have collapsed spaces; prefer taking the span that corresponds to raw_group if present
            if raw_group:
                # find raw_group starting at idx or after
                rg_idx = orig.find(raw_group, idx)
                if rg_idx != -1 and rg_idx >= idx and rg_idx < end + 20:
                    idx = rg_idx
                    end = rg_idx + len(raw_group)
            match_text_orig = orig[idx:end]
            results.append({
                'pattern': name,
                'matched_normalized': matched_norm,
                'raw_group': raw_group,
                'mapped': True,
                'orig_index': idx,
                'orig_end': end,
                'orig_match_text': match_text_orig
            })
            # move cursor to end to avoid duplicate finds
            orig_cursor = end
    return results

# 5) Run debug on first N records to show per-record pattern behavior
N = 50
hits = []
with open(JSONL, 'r', encoding='utf-8') as fin:
    for i, line in enumerate(fin):
        if i >= N: break
        rec = json.loads(line)
        text = rec.get('text') or rec.get('sentence') or rec.get('line') or rec.get('content') or ''
        res = find_matches_map_to_original(text, patterns=patterns)
        # collect only mapped successes
        mapped = [r for r in res if r.get('mapped')]
        if mapped:
            print(f"\n=== LINE #{i} page={rec.get('page')} line_id={rec.get('line_id')} ===")
            print("ORIG repr:", repr(text[:200]))
            for m in mapped:
                print("pattern:", m['pattern'], "orig_index:", m['orig_index'], "orig_end:", m['orig_end'], "match_text:", repr(m['orig_match_text']))
            hits.append((i, mapped))
print("\nTOTAL LINES with mapped hits in first", N, "=", len(hits))

# 6) If mapped hits found, write a CSV using the robust mapper logic (global offsets if start present)
if len(hits) > 0:
    import csv
    out_rows = []
    with open(JSONL, 'r', encoding='utf-8') as fin:
        for i, line in enumerate(fin):
            rec = json.loads(line)
            text = rec.get('text') or rec.get('sentence') or rec.get('line') or rec.get('content') or ''
            page = rec.get('page')
            line_id = rec.get('line_id')
            sent_start = rec.get('start_char') or rec.get('start') or rec.get('char_start') or 0
            for m in find_matches_map_to_original(text, patterns=patterns):
                if not m.get('mapped'):
                    continue
                out_rows.append({
                    "line_no": i,
                    "page": page,
                    "line_id": line_id,
                    "sentence_text": text,
                    "match_text": m['orig_match_text'],
                    "match_start": m['orig_index'],
                    "match_end": m['orig_end'],
                    "global_start": (sent_start + m['orig_index']) if isinstance(sent_start, int) else None,
                    "global_end": (sent_start + m['orig_end']) if isinstance(sent_start, int) else None,
                    "raw_group": m['raw_group'],
                    "pattern": m['pattern']
                })
    # write CSV
    with open(OUT_CSV, 'w', encoding='utf-8', newline='') as fout:
        w = csv.DictWriter(fout, fieldnames=list(out_rows[0].keys()))
        w.writeheader()
        for r in out_rows:
            w.writerow(r)
    print("Wrote debug CSV:", OUT_CSV, "rows:", len(out_rows))
else:
    print("No mapped matches found in first", N, "records. Please paste one example sentence that definitely contains 'Section' or 'IPC' (copy its exact text) and I will craft a custom pattern.")

JSONL exists? True

First record keys: ['page', 'line_id', 'start_char', 'end_char', 'text']

Sample text (repr):
 'Purushottam Dashrath Borate vs State Of Maharashtra on 8\nMay, 2015\nAuthor: H.L.'

Sample text (print):
 Purushottam Dashrath Borate vs State Of Maharashtra on 8
May, 2015
Author: H.L.

Char codes (first 120 chars):
0 P 0x50
1 u 0x75
2 r 0x72
3 u 0x75
4 s 0x73
5 h 0x68
6 o 0x6f
7 t 0x74
8 t 0x74
9 a 0x61
10 m 0x6d
11   0x20
12 D 0x44
13 a 0x61
14 s 0x73
15 h 0x68
16 r 0x72
17 a 0x61
18 t 0x74
19 h 0x68
20   0x20
21 B 0x42
22 o 0x6f
23 r 0x72
24 a 0x61
25 t 0x74
26 e 0x65
27   0x20
28 v 0x76
29 s 0x73
30   0x20
31 S 0x53
32 t 0x74
33 a 0x61
34 t 0x74
35 e 0x65
36   0x20
37 O 0x4f
38 f 0x66
39   0x20
40 M 0x4d
41 a 0x61
42 h 0x68
43 a 0x61
44 r 0x72
45 a 0x61
46 s 0x73
47 h 0x68
48 t 0x74
49 r 0x72
50 a 0x61
51   0x20
52 o 0x6f
53 n 0x6e
54   0x20
55 8 0x38
56 
 0xa
57 M 0x4d
58 a 0x61
59 y 0x79
60 , 0x2c
61   0x20
62 2 0x32
63 0 0x30
64 1 0x31
65 5 0x35
66 
 0xa
67 A 0x41

runn regex mapper

In [19]:
# Run mapper and write CSV
from src import regex_mapper
jsonl_path = "/content/casetext.jsonl"   # update if your file is different
out_csv = "/content/regex_hits_with_global_offsets.csv"

rows = regex_mapper.map_jsonl_file_to_hits(jsonl_path, out_csv, include_legal_terms=True)
print("Total hits:", len(rows))

Wrote 0 hits to /content/regex_hits_with_global_offsets.csv
Total hits: 0


In [28]:
# ========= CORRECT canon mapping for IDs like IPC_120, IPC_120B, IPC_376_2_G ==========
import pandas as pd
import re
from fuzzywuzzy import fuzz, process
from collections import defaultdict

# Paths (update if needed)
HITS_CSV = "/content/regex_hits_debug.csv"
CANONICAL_CSV = "/content/LegalTalk/data/statutes.csv"

OUT_WEAK = "/content/weak_labels.csv"
OUT_MULTIHOT = "/content/weak_labels_multihot.csv"

# Load datasets
hits = pd.read_csv(HITS_CSV, dtype=str).fillna('')
canon = pd.read_csv(CANONICAL_CSV, dtype=str).fillna('')

# Canonical IDs (true IDs) like IPC_120, IPC_376, IPC_120B
canonical_ids = set(canon['id'].astype(str).tolist())

# ---------- Utility functions ----------
def normalize_canonical_id(raw: str):
    """Convert raw '120-B', '376(2)(g)' to canonical IPC format: IPC_120B, IPC_376_2_G."""
    if not raw:
        return None

    s = raw.upper().strip()

    # Remove "SECTION", "SEC", "S." etc.
    s = re.sub(r'^(SECTION|SEC|S|SS)\.?\s*', "", s)

    # Remove 'IPC_' prefix if present (we will add it later)
    s = s.replace("IPC_", "").replace("IPC", "").strip()

    # Case 1: simple number or number+letter like 120, 120B
    if re.match(r'^\d{1,4}[A-Z]?$' , s):
        return f"IPC_{s}"

    # Case 2: Patterns like 376(2)(g)
    if "(" in s:
        # Replace parentheses (2)(g) -> _2_G
        s2 = re.sub(r'\((\d+)\)', r'_\1', s)
        s2 = re.sub(r'\(([A-Z])\)', r'_\1', s2)
        s2 = s2.replace("-", "").replace(" ", "")
        return "IPC_" + s2

    # Hyphens in 120-B -> 120B
    s = s.replace("-", "")

    # Final fallback
    return "IPC_" + s


def map_to_canonical(stat_norm: str, canon_df=canon):
    """
    Convert normalized regex match (e.g., '120B', '302')
    to canonical ID (e.g., 'IPC_120B', 'IPC_302').
    """
    if not stat_norm or stat_norm.strip()=="": # Corrected the condition here
        return None

    candidate = normalize_canonical_id(stat_norm)

    # Direct hit
    if candidate in canonical_ids:
        return candidate

    # Try removing trailing letters (e.g., 120B → 120)
    digits = re.match(r'^(\d+)', candidate.replace("IPC_", ""))
    if digits:
        basic = f"IPC_{digits.group(1)}"
        if basic in canonical_ids:
            return basic

    # Try fuzzy match against canonical list
    best = process.extractOne(candidate, list(canonical_ids), scorer=fuzz.token_set_ratio)
    if best and best[1] > 90:
        return best[0]

    return None


# ---------- Build weak-label dataset ----------
grouped = defaultdict(set)
meta = {}

for _, row in hits.iterrows():
    key = f"{row.get('page','')}__{row.get('line_id','')}"
    stat_norm = row.get('statute_normalized', '') or row.get('raw_group','')

    cid = map_to_canonical(stat_norm)

    if cid:
        grouped[key].add(cid)

    meta[key] = {
        'page': row.get('page',''),
        'line_id': row.get('line_id',''),
        'sentence_text': row.get('sentence_text','')
    }

# Build DataFrame
rows = []
for k, values in grouped.items():
    m = meta[k]
    rows.append({
        'sentence_id': k,
        'page': m['page'],
        'line_id': m['line_id'],
        'sentence_text': m['sentence_text'],
        'canonical_statutes': ";".join(sorted(values))
    })

weak_df = pd.DataFrame(rows)
weak_df.to_csv(OUT_WEAK, index=False)
print("Wrote weak_labels.csv →", len(weak_df), "rows")

# ---------- Optional multi-hot ----------
all_canon_sorted = sorted(list(canonical_ids))

for cid in all_canon_sorted:
    weak_df[cid] = weak_df['canonical_statutes'].apply(lambda s: 1 if (cid in str(s).split(";")) else 0)

weak_df.to_csv(OUT_MULTIHOT, index=False)
print("Wrote multihot:", OUT_MULTIHOT, "shape:", weak_df.shape)

# Show top rows
weak_df.head(10)
!pip install fuzzywuzzy

Wrote weak_labels.csv → 11 rows
Wrote multihot: /content/weak_labels_multihot.csv shape: (11, 459)


/tmp/ipython-input-735299990.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weak_df[cid] = weak_df['canonical_statutes'].apply(lambda s: 1 if (cid in str(s).split(";")) else 0)
/tmp/ipython-input-735299990.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weak_df[cid] = weak_df['canonical_statutes'].apply(lambda s: 1 if (cid in str(s).split(";")) else 0)
/tmp/ipython-input-735299990.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

In [25]:
!pip install fuzzywuzzy

In [22]:
# 1) Inspect the CSV you produced
import pandas as pd
from pathlib import Path
from src.utils import normalize_to_ipc
import inspect, importlib, src.regex_mapper as rm

CSV_PATH = "/content/regex_hits3.csv"   # update to the file you actually wrote
if not Path(CSV_PATH).exists():
    # try other common names
    for alt in ["/content/regex_hits.csv", "/content/regex_hits_debug.csv", "/content/regex_hits_fixed.csv"]:
        if Path(alt).exists():
            CSV_PATH = alt
            break

print("Inspecting:", CSV_PATH)
df = pd.read_csv(CSV_PATH, dtype=str).fillna('')
print("\nColumns found:", df.columns.tolist())
print("\nFirst 5 rows:")
display(df.head(5))

# 2) If canonical_id missing, create it now from statute_raw or statute_normalized
if 'canonical_id' not in df.columns:
    # choose source column
    src_col = None
    for c in ['statute_raw','statute_normalized','statute','raw_id','match_text']:
        if c in df.columns:
            src_col = c
            break
    if src_col is None:
        raise ValueError("No plausible statute source column found in CSV. Columns: " + ", ".join(df.columns.tolist()))
    print(f"\nCanonical_id not found. Will compute from column: {src_col}")

    # compute canonical_id using normalize_to_ipc (imported from src.utils)
    df['canonical_id'] = df[src_col].apply(lambda x: normalize_to_ipc(x) if pd.notna(x) else None)

    # Report how many None values (unmapped)
    n_none = df['canonical_id'].isna().sum()
    print(f"Computed canonical_id. Unmapped/None count: {n_none} / {len(df)}")

    # Save fixed CSV
    out_fixed = Path(CSV_PATH).with_name(Path(CSV_PATH).stem + "_fixed.csv")
    df.to_csv(out_fixed, index=False)
    print("Wrote fixed CSV to", out_fixed)
    display(df.head(10))
else:
    print("\ncanonical_id column already present. No action needed.")

# 3) (Optional) If you want to re-run the mapping function to regenerate CSV from JSONL,
#    ensure you reload the module to pick up any edits and call it again.
print("\nFunction map_jsonl_file_to_hits docstring (to verify you're using the updated version):\n")
importlib.reload(rm)
print(inspect.getsource(rm.map_jsonl_file_to_hits)[:800])  # show beginning of function

# Example to re-run mapping (uncomment and update paths if you want to regenerate)
# jsonl_path = "/content/casetext.jsonl"
# out_csv = "/content/regex_hits_with_canonical.csv"
# rows = rm.map_jsonl_file_to_hits(jsonl_path, out_csv, filter_none_canonical=False)
# print("Rows returned:", len(rows))

Inspecting: /content/regex_hits3.csv

Columns found: ['line_no', 'page', 'line_id', 'sentence_text', 'statute_raw', 'pattern_name', 'match_text', 'local_start', 'local_end', 'global_start', 'global_end']

First 5 rows:


,line_no,page,line_id,sentence_text,statute_raw,pattern_name,match_text,local_start,local_end,global_start,global_end
0,8,0,8,"By the impugned judgment and order,\nthe High ...",302,section_keyword,Sections 302,312,315,1359,1362
1,8,0,8,"By the impugned judgment and order,\nthe High ...",120-B,section_keyword,Section 120-B,358,363,1405,1410
2,8,0,8,"By the impugned judgment and order,\nthe High ...",1860,ipc_prefix,"Indian Penal Code, 1860",390,394,1437,1441
3,8,0,8,"By the impugned judgment and order,\nthe High ...",120-B,section_of_ipc,Section 120-B of the Indian Penal Code,358,363,1405,1410
4,43,2,5,"Therefore, an offence under Section\n302 of th...",302,section_keyword,Section 302,36,39,383,386



Canonical_id not found. Will compute from column: statute_raw
Computed canonical_id. Unmapped/None count: 1 / 36
Wrote fixed CSV to /content/regex_hits3_fixed.csv


,line_no,page,line_id,sentence_text,statute_raw,pattern_name,match_text,local_start,local_end,global_start,global_end,canonical_id
0,8,0,8,"By the impugned judgment and order,\nthe High ...",302,section_keyword,Sections 302,312,315,1359,1362,IPC_302
1,8,0,8,"By the impugned judgment and order,\nthe High ...",120-B,section_keyword,Section 120-B,358,363,1405,1410,IPC_120B
2,8,0,8,"By the impugned judgment and order,\nthe High ...",1860,ipc_prefix,"Indian Penal Code, 1860",390,394,1437,1441,None
3,8,0,8,"By the impugned judgment and order,\nthe High ...",120-B,section_of_ipc,Section 120-B of the Indian Penal Code,358,363,1405,1410,IPC_120B
4,43,2,5,"Therefore, an offence under Section\n302 of th...",302,section_keyword,Section 302,36,39,383,386,IPC_302
5,43,2,5,"Therefore, an offence under Section\n302 of th...",302,section_of_ipc,Section 302 of the IPC,36,39,383,386,IPC_302
6,69,2,31,"On 05.03.2009, the\ncharges were framed under ...",364,section_keyword,Sections 364,54,57,3331,3334,IPC_364
7,70,2,32,"On 03.04.2010, the charge was altered and the ...",120-B,section_keyword,Section 120-B,93,98,3493,3498,IPC_120B
8,70,2,32,"On 03.04.2010, the charge was altered and the ...",120-B,section_of_ipc,Section 120-B of the IPC,93,98,3493,3498,IPC_120B
9,71,2,33,"Additionally, the charge under Section 120-B o...",120-B,section_keyword,Section 120-B,39,44,3560,3565,IPC_120B



Function map_jsonl_file_to_hits docstring (to verify you're using the updated version):

def map_jsonl_file_to_hits(jsonl_path, out_csv, include_legal_terms=True):
    """
    Reads JSONL lines of:
        { "page":..., "line_id":..., "start_char":..., "end_char":..., "text":... }
    Extracts statute references into CSV.
    """

    jsonl_path = str(jsonl_path)
    out_csv = str(out_csv)

    patterns = build_statute_patterns()
    rows = []

    with open(jsonl_path, "r", encoding="utf-8") as fin:
        for line_no, line in enumerate(fin):
            rec = json.loads(line)
            txt = rec.get("text", "") or ""

            page = rec.get("page")
            line_id = rec.get("line_id")
            start_char = rec.get("start_char")

            hits = find_statutes_in_text(txt, patterns=patterns)

            for h in hits:
              statute_raw = h["statute_ra
